In [1]:
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups(subset="train")
print("Number of text samples {}".format(len(data.data)))

Number of text samples 11314


In [2]:
import pandas as pd
df = pd.DataFrame()
df = df.assign(text=data["data"]).assign(target=data["target"])
df.head()

,text,target
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14


In [3]:
import re
def remove_emails(text):
    '''
    Remove any email address in text with
    
    The `regex` can catch any number of email addresses in the text. Regex can be tried here: https://regex101.com/r/ZjgyLc/2
    '''
    regex =  r'\S*@\S*\s?'
    return re.sub(regex, '', text)
test_text = remove_emails(data.data[0])

In [4]:
def remove_newlinechars(text):
    '''
    Substitute any newline chars with a whitespach
    
    The `regex` can be tried at: https://regex101.com/r/2fImPz/1/
    '''
    regex = r'\s+'
    return re.sub(regex, ' ', text)
test_text = remove_newlinechars(test_text)
print(test_text)

From: (where's my thing) Subject: WHAT car is this!? Nntp-Posting-Host: rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: 15 I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2-door sports car, looked to be from the late 60s/ early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail. Thanks, - IL ---- brought to you by your neighborhood Lerxst ---- 


In [5]:
import nltk
def tokenize(text):
    '''
    Tokenize text
    '''
    tokens = nltk.word_tokenize(text)
    
    return list(
        filter(lambda word: word.isalnum(), tokens)
    )
test_text = tokenize(test_text)
print(test_text)

['From', 'where', 'my', 'thing', 'Subject', 'WHAT', 'car', 'is', 'this', 'Organization', 'University', 'of', 'Maryland', 'College', 'Park', 'Lines', '15', 'I', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'I', 'saw', 'the', 'other', 'day', 'It', 'was', 'a', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', '70s', 'It', 'was', 'called', 'a', 'Bricklin', 'The', 'doors', 'were', 'really', 'small', 'In', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'This', 'is', 'all', 'I', 'know', 'If', 'anyone', 'can', 'tellme', 'a', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'Thanks', 'IL', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'Lerxst']


In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
## Add some common words from text

stop_words.extend(["from","subject","summary","keywords","article"])

def remove_stopwords(words):
    filtered = filter(lambda word: word not in stop_words,words)
    return list(filtered)
    
test_text = remove_stopwords(test_text)
print(test_text)

['From', 'thing', 'Subject', 'WHAT', 'car', 'Organization', 'University', 'Maryland', 'College', 'Park', 'Lines', '15', 'I', 'wondering', 'anyone', 'could', 'enlighten', 'car', 'I', 'saw', 'day', 'It', 'sports', 'car', 'looked', 'late', 'early', '70s', 'It', 'called', 'Bricklin', 'The', 'doors', 'really', 'small', 'In', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'This', 'I', 'know', 'If', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'production', 'car', 'made', 'history', 'whatever', 'info', 'funky', 'looking', 'car', 'please', 'Thanks', 'IL', 'brought', 'neighborhood', 'Lerxst']


In [7]:
import spacy
from spacy.lang.en.examples import sentences 

nlp = spacy.load("en_core_web_sm")

def lemmatize(text, nlp=nlp):
    
    doc = nlp(" ".join(text))
    
    lemmatized = [token.lemma_ for token in doc]
    
    return lemmatized
test_text = lemmatize(test_text,nlp)
print(test_text)

['from', 'thing', 'Subject', 'what', 'car', 'Organization', 'University', 'Maryland', 'College', 'Park', 'Lines', '15', '-PRON-', 'wonder', 'anyone', 'could', 'enlighten', 'car', '-PRON-', 'see', 'day', '-PRON-', 'sport', 'car', 'look', 'late', 'early', '70', '-PRON-', 'call', 'Bricklin', 'the', 'door', 'really', 'small', 'in', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'this', '-PRON-', 'know', 'if', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'year', 'production', 'car', 'make', 'history', 'whatever', 'info', 'funky', 'look', 'car', 'please', 'thank', 'IL', 'bring', 'neighborhood', 'lerxst']


In [8]:
import time
t0 = time.time()
def clean_text(df):
    '''
    Take in a Dataframe, and process it
    '''
    df["cleaned_text"] = df.text.map(lambda text:text.lower()).map(remove_emails).map(remove_newlinechars).map(remove_stopwords).map(lemmatize)
    return df
#df = clean_text(df)
t1 = time.time()
print("Time to process without Dask {}".format(t1-t0))

KeyboardInterrupt: 

In [11]:
import dask.dataframe as ddf
dask_dataframe = ddf.from_pandas(df, npartitions=600)

In [13]:
t0 = time.time()
result = dask_dataframe.map_partitions(clean_text, meta=df)
dg = result.compute()
t1 = time.time()
print("Time to process with Dask {}".format(t1-t0))

C:\Users\clyde\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


ValueError: The columns in the computed data do not match the columns in the provided metadata